In [1]:
import numpy as np
import math
import itertools
import time
from multiprocessing import Pool
import scipy.sparse

In [2]:
#Given an omino (nxn binary matrix), checks if it is valid
# i.e. all ones are rook-adjacent and contiguous

def valid_omino(omino):
    k = omino.shape[0]
    check = np.zeros([k,k]) - omino
    
    
    _f  = False
    for i in range(k):
        for j in range(k):
            if check[i,j] == -1:
                if not _f:
                    _f  = True
                    check[i,j] = 1
    if not _f:
        print("ABORT")
        return
    
    for iter in range(15):
        for i in range(k):
            for j in range(k):
                if check[i,j] == 1:
                    if i != 0:
                        check[i-1,j] = check[i-1,j]**2


                    if i != k-1:
                        check[i+1,j] = check[i+1,j]**2

                    if j != 0:
                        check[i,j-1] = check[i,j-1]**2

                    if j != k-1:
                        check[i,j+1] = check[i,j+1]**2
                    
    return np.sum(check) == np.sum(omino)
                        
    



In [3]:
# given a list of (r,c) index pairs, generates an omino
# if the omino isn't valid, returns the 0x0 0 matrix
def make_omino(inds):
    dim = len(inds)
    p = np.zeros([dim,dim])
    for loc in inds:
        p[loc[0],loc[1]] = 1
    
    if valid_omino(p):
        return p
    else:
        return np.zeros([0,0])

In [4]:
# generates all of the <cells>-ominos in the <grid>^2 grid
# WARNING - not tested for cells != grid
def make_omino_set(cells, grid):
    
    
    if grid % cells != 0:
        print("WARNING - <cells> must divide <grid>")
        return []
    
    
    pair_idx = []
    for i in range(grid):
        for j in range(grid):
            pair_idx.append((i,j))
    
    
    ominos = []
    
    for t in (l for l in itertools.combinations(pair_idx, cells)):
        p = make_omino(t)
        if p.shape[0] != 0:
            ominos.append(p)
    
    return ominos
        

In [5]:
# make the 4x4 ominos
n = make_omino_set(4,4)
print(len(n))
print([np.sum(x) for x in n if np.sum(x) != 4])


113
[]


In [ ]:
# check them
counter = 0
found = 0
ones = np.ones([4,4])
print(ones)
val = []
t = time.time()
for part in itertools.combinations(n,4):

    counter += 1
    if np.prod(sum(part)) == 1:
        found +=1
        val.append(part)


np.save("parts_4-4", val)        
print("FOUND: " + str(found))
print(time.time() - t)

In [ ]:
# do the 5x5
n = make_omino_set(5,5)
print(len(n))
print([np.sum(x) for x in n if np.sum(x) != 5])

In [ ]:
# build up to pairs and triples and 4s
counter = 0
found = 0
ones = np.ones([5,5])
print(ones)
val = []
t = time.time()

pairs = [x for x in itertools.combinations(n,2) if 2 not in sum(x)]
trips = []
counter = 0
for p in pairs:
    for o in n:
        counter+=1
        if counter % 1000000 == 0: print(counter/1000000)
        t = p + (o,)
        if 2 not in sum(t):
            trips.append(t)
            
counter = 0
quads = []
for t in trips:
    for o in n:
        counter +=1
        if counter % 1000000 == 0: print(counter/1000000)
        q = t + (o,)
        if 2 not in sum(q):
            quads.append(q)

In [ ]:
# build 4s         
print(len(quads))
found = 0
valid_quads = []
counter = 0
for q in quads:
    counter += 1
    if counter % 100000 == 0:
        print(counter/100000, found)
    if valid_omino(ones-sum(q)):
        valid_quads.append(q)
        found += 1
    

In [ ]:
# build the valid partitions and check uniqueness
partitions = [ q + (ones-sum(q),) for q in valid_quads  ]
print(len(partitions))
np.save("parts_5-5",partitions)
mat = np.load("parts_5-5.npy")
mat2 = np.sort(mat)
counter = 0
print(len(mat),len(mat2), len(mat3))
final_parts = []
for x in mat2:
    counter +=1
    if counter%10000 ==0: print(counter/10000)
    if len([y for y in final_parts if np.array_equiv(x,y)]) ==0:
        final_parts.append(x)
np.save("final_parts_5-5", final_parts)